# Download all endogenous data (fred_data) and target data (NFP_target) using ALFRED

In [18]:
import sys
import logging
import importlib
from pathlib import Path

# 1. Ensure project root is in python path
# This assumes your notebook is in the root folder (parent of Load_Data)
current_dir = Path.cwd()
if str(current_dir) not in sys.path:
    sys.path.append(str(current_dir))

# 2. Reload modules to ensure you are using the latest code edits
import Load_Data.fred_snapshots
importlib.reload(Load_Data.fred_snapshots)

# 3. Import the function and settings
from Load_Data.fred_snapshots import build_all_snapshots
from settings import START_DATE, END_DATE

# 4. Configure Logging (Force it to print to notebook output)
# We remove existing handlers to prevent duplicate logs
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    force=True
)

# 5. Run the Build
# refresh_existing=True will force it to re-process monthly snapshots even if they exist.
# use_disk_cache=True allows it to use the downloaded audit file if you have it, saving time.
print(f"🚀 Starting FRED Snapshot Build from {START_DATE} to {END_DATE}...")

build_all_snapshots(
    start_date=START_DATE, 
    end_date=END_DATE, 
    refresh_existing=True  # Set to True to overwrite existing snapshot files
)

print("✅ Done.")

2025-12-16 02:18:36,046 - fred_snapshots - INFO - Downloading ALL 258 series AS OF 2025-09-30


🚀 Starting FRED Snapshot Build from 1990-01-01 to 2025-09-30...


2025-12-16 02:18:37,827 - fred_snapshots - INFO - total: Fixed 144 observations with data quality issues (using First Friday of next month rule)
2025-12-16 02:18:38,964 - fred_snapshots - INFO - total_nsa: Fixed 717 observations with data quality issues (using First Friday of next month rule)
2025-12-16 02:18:40,895 - fred_snapshots - INFO - total.private: Fixed 273 observations with data quality issues (using First Friday of next month rule)
2025-12-16 02:18:42,040 - fred_snapshots - INFO - total.private_nsa: Fixed 747 observations with data quality issues (using First Friday of next month rule)
2025-12-16 02:18:43,329 - fred_snapshots - INFO - total.government: Fixed 155 observations with data quality issues (using First Friday of next month rule)
2025-12-16 02:18:44,286 - fred_snapshots - INFO - total.government_nsa: Fixed 747 observations with data quality issues (using First Friday of next month rule)
2025-12-16 02:18:45,810 - fred_snapshots - INFO - total.private.goods: Fixed 273

✅ Done.


# Build our exogenous features to use for NBEATSx model

In [4]:
import logging
import sys
import shutil
import importlib
from pathlib import Path

# 1. Setup Path
current_dir = Path.cwd()
if str(current_dir) not in sys.path:
    sys.path.append(str(current_dir))

# 2. Import Settings
import settings
# Reload settings in case dates changed
importlib.reload(settings)
from settings import START_DATE, END_DATE, DATA_PATH

# ------------------------------------------------------------------
# MODULE RELOADER (Ensures you use your latest code edits)
# ------------------------------------------------------------------
# If you just edited these files, we must force Python to reload them
import Load_Data.load_fred_exogenous
import Load_Data.load_unifier_data
import Load_Data.load_noaa_data
import Prepare_Data.create_noaa_master
import Prepare_Data.create_noaa_weighted
import Prepare_Data.create_adp_snapshots
import Prepare_Data.create_master_snapshots

importlib.reload(Load_Data.load_fred_exogenous)
importlib.reload(Load_Data.load_unifier_data)
importlib.reload(Load_Data.load_noaa_data)
importlib.reload(Prepare_Data.create_noaa_master)
importlib.reload(Prepare_Data.create_noaa_weighted)
importlib.reload(Prepare_Data.create_adp_snapshots)
importlib.reload(Prepare_Data.create_master_snapshots)

# Now import the functions from the reloaded modules
from Load_Data.load_fred_exogenous import fetch_fred_exogenous_snapshots
from Load_Data.load_unifier_data import fetch_unifier_snapshots
from Load_Data.load_noaa_data import load_noaa_data
from Prepare_Data.create_noaa_master import create_noaa_master
from Prepare_Data.create_noaa_weighted import create_noaa_weighted_snapshots
from Prepare_Data.create_adp_snapshots import create_adp_snapshots
from Prepare_Data.create_master_snapshots import create_master_snapshots, verify_master_snapshot
'''
# ------------------------------------------------------------------
# LOGGING CONFIGURATION
# ------------------------------------------------------------------
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(message)s',
    force=True
)
logger = logging.getLogger()

# Mute noisy internal loggers
logging.getLogger("Load_Data.load_fred_exogenous").setLevel(logging.WARNING)
logging.getLogger("load_fred_exogenous").setLevel(logging.WARNING)
logging.getLogger("fredapi").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("Prepare_Data.create_master_snapshots").setLevel(logging.WARNING)

# ------------------------------------------------------------------
# DATA CLEANING (The "Nuclear Option")
# ------------------------------------------------------------------
def clean_slate():
    """Deletes all generated data folders to force a full rebuild."""
    logger.info("🧹 STARTING CLEAN SLATE PROTOCOL...")
    
    # Folders to DELETE (Everything except raw ADP)
    paths_to_purge = [
        DATA_PATH / "Exogenous_data" / "exogenous_fred_data",
        DATA_PATH / "Exogenous_data" / "exogenous_unifier_data",
        DATA_PATH / "Exogenous_data" / "noaa_weighted_snapshots",
        DATA_PATH / "Exogenous_data" / "master_snapshots",
        DATA_PATH / "Exogenous_data" / "ADP_snapshots", # Delete snapshots, keep raw ADP
        DATA_PATH / "Exogenous_data" / "NOAA_data"      # Delete state files to force redownload
    ]
    
    for p in paths_to_purge:
        if p.exists():
            logger.info(f"   🗑️ Deleting: {p}")
            shutil.rmtree(p)
        else:
            logger.info(f"   Note: {p} did not exist (skipping delete).")
            
    logger.info("✨ Clean slate complete. Ready to rebuild.\n")

# ------------------------------------------------------------------
# PIPELINE EXECUTION
# ------------------------------------------------------------------

# 1. Run Cleaner
clean_slate()

logger.info("🚀 STARTING FULL DATA PIPELINE REGENERATION")
logger.info(f"📅 Date Range: {START_DATE} to {END_DATE}")
logger.info("⚠️  Skipping ADP Web Scraping (using existing CSV).")

# =========================================================
# PHASE 1: DOWNLOAD / REFRESH EXTERNAL DATA
# =========================================================

logger.info("\n--- [1/7] Fetching FRED Data (Financials, JOLTS, Claims) ---")
fetch_fred_exogenous_snapshots(start_date=START_DATE, end_date=END_DATE)
logger.info("   ✓ FRED Data Fetch Complete")

logger.info("\n--- [2/7] Fetching Unifier Data (ISM, Challenger, Consumer Conf) ---")
fetch_unifier_snapshots(start_date=START_DATE, end_date=END_DATE)
logger.info("   ✓ Unifier Data Fetch Complete")

logger.info("\n--- [3/7] Fetching NOAA Storm Data (FTP Download & State Aggregation) ---")
logger.info("   (This may take a few minutes...)")
load_noaa_data()
logger.info("   ✓ NOAA Data Fetch Complete")

# =========================================================
# PHASE 2: PROCESS & TRANSFORM
# =========================================================

logger.info("\n--- [4/7] Creating NOAA Master File (Merging States to US) ---")
create_noaa_master()
logger.info("   ✓ NOAA Master File Created")

logger.info("\n--- [5/7] Creating NOAA Weighted Snapshots ---")
logger.info("    -> Applying NFP Weights")
logger.info("    -> CRITICAL: Applying Log1p Transform here")
create_noaa_weighted_snapshots(start_date=START_DATE, end_date=END_DATE)
logger.info("   ✓ NOAA Weighted Snapshots Created")

logger.info("\n--- [6/7] Regenerating ADP Snapshots ---")
# Uses existing 'ADP_Employment_Change.parquet' on disk
create_adp_snapshots()
logger.info("   ✓ ADP Snapshots Created")

# =========================================================
# PHASE 3: MASTER MERGE
# =========================================================

logger.info("\n--- [7/7] Creating Master Snapshots ---")
logger.info("    -> Merging FRED, ADP, Unifier, NOAA")
logger.info("    -> Calculating Rolling Means")
logger.info("    -> Generating Vintage Lags")
create_master_snapshots()
logger.info("   ✓ Master Snapshots Created")

logger.info("\n--- Verifying Final Output ---")
# Re-enable logging for verification
logging.getLogger("Prepare_Data.create_master_snapshots").setLevel(logging.INFO)
verify_master_snapshot()

logger.info("\n✅ PIPELINE COMPLETE!")
'''
create_master_snapshots()


INFO:create_master_snapshots:Generating Cleaned Master Snapshots from 1990-01-01 to 2025-09-30
INFO:create_master_snapshots:Generated Snapshot for 1991-01-31
INFO:create_master_snapshots:Generated Snapshot for 1992-01-31
INFO:create_master_snapshots:Generated Snapshot for 1993-01-31
INFO:create_master_snapshots:Generated Snapshot for 1994-01-31
INFO:create_master_snapshots:Generated Snapshot for 1995-01-31
INFO:create_master_snapshots:Generated Snapshot for 1996-01-31
INFO:create_master_snapshots:Generated Snapshot for 1997-01-31
INFO:create_master_snapshots:Generated Snapshot for 1998-01-31
INFO:create_master_snapshots:Generated Snapshot for 1999-01-31
INFO:create_master_snapshots:Generated Snapshot for 2000-01-31
INFO:create_master_snapshots:Generated Snapshot for 2001-01-31
INFO:create_master_snapshots:Generated Snapshot for 2002-01-31
INFO:create_master_snapshots:Generated Snapshot for 2003-01-31
INFO:create_master_snapshots:Generated Snapshot for 2004-01-31
INFO:create_master_snap